In [3]:
import pandas as pd
df=pd.read_csv('/content/imdb_dataset.csv')
df.head()

,text,label,type
0,Story of a man who has unnatural feelings for ...,0,train
1,Airport '77 starts as a brand new luxury 747 p...,0,train
2,This film lacked something I couldn't put my f...,0,train
3,"Sorry everyone,,, I know this is supposed to b...",0,train
4,When I was little my parents took me along to ...,0,train


In [4]:
import re
import html
import unicodedata

def clean_text_bert(text):
    text = html.unescape(str(text))
    text = re.sub(r'<.*?>', ' ', text)
    text = unicodedata.normalize("NFKC", text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'([,.!?])\1+', r'\1', text)  # thu gọn dấu câu lặp
    text = text.lower()  # nếu bạn dùng bert-base-uncased
    return text


In [5]:
train_df = df[df['type'] == 'train'].copy()
test_df  = df[df['type'] == 'test'].copy()

print(len(train_df), len(test_df))


25000 25000


In [7]:
train_df.head()

,text,label,type
0,Story of a man who has unnatural feelings for ...,0,train
1,Airport '77 starts as a brand new luxury 747 p...,0,train
2,This film lacked something I couldn't put my f...,0,train
3,"Sorry everyone,,, I know this is supposed to b...",0,train
4,When I was little my parents took me along to ...,0,train


In [6]:
bert_data = train_df['text'].apply(clean_text_bert)

In [8]:
print(bert_data)

0        story of a man who has unnatural feelings for ...
1        airport '77 starts as a brand new luxury 747 p...
2        this film lacked something i couldn't put my f...
3        sorry everyone, i know this is supposed to be ...
4        when i was little my parents took me along to ...
                               ...                        
24995    seeing as the vote average was pretty low, and...
24996    the plot had some wretched, unbelievable twist...
24997    i am amazed at how this movie(and most others ...
24998    a christmas together actually came before my t...
24999    working-class romantic drama from director mar...
Name: text, Length: 25000, dtype: object


In [9]:
bert_data.head()

,text
0,story of a man who has unnatural feelings for ...
1,airport '77 starts as a brand new luxury 747 p...
2,this film lacked something i couldn't put my f...
3,"sorry everyone, i know this is supposed to be ..."
4,when i was little my parents took me along to ...


In [10]:
bert_df = pd.DataFrame({
    'text': bert_data,
    'label': train_df['label']
})


In [12]:
bert_df.shape

(25000, 2)

In [13]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install transformers scikit-learn tqdm


Looking in indexes: https://download.pytorch.org/whl/cu121


In [14]:

from sklearn.model_selection import StratifiedKFold
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
import torch
import numpy as np
from tqdm import tqdm
from torch.optim import AdamW
from transformers.optimization import get_linear_schedule_with_warmup

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
accuracies = []

for fold, (train_idx, val_idx) in enumerate(skf.split(bert_df['text'], bert_df['label'])):
    print(f"\n===== Fold {fold+1} =====")

    X_train, X_val = bert_df['text'].iloc[train_idx], bert_df['text'].iloc[val_idx]
    y_train, y_val = bert_df['label'].iloc[train_idx], bert_df['label'].iloc[val_idx]

    # 3Tokenize
    train_enc = tokenizer(list(X_train), truncation=True, padding=True, max_length=128, return_tensors='pt')
    val_enc   = tokenizer(list(X_val), truncation=True, padding=True, max_length=128, return_tensors='pt')

    # 4 Dataset & DataLoader
    train_dataset = torch.utils.data.TensorDataset(
        train_enc['input_ids'], train_enc['attention_mask'], torch.tensor(y_train.values)
    )
    val_dataset = torch.utils.data.TensorDataset(
        val_enc['input_ids'], val_enc['attention_mask'], torch.tensor(y_val.values)
    )

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=16)

    # 5️Model + Optimizer
    model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
    model.to(device)

    optimizer = AdamW(model.parameters(), lr=2e-5)
    total_steps = len(train_loader) * 3
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
    # 6️ Huấn luyện
    for epoch in range(3):
        model.train()
        total_loss = 0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
            b_input_ids, b_mask, b_labels = [t.to(device) for t in batch]
            optimizer.zero_grad()

            outputs = model(input_ids=b_input_ids, attention_mask=b_mask, labels=b_labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            scheduler.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1} | Loss: {total_loss/len(train_loader):.4f}")

    # 7️ Đánh giá
    model.eval()
    preds, true_labels = [], []

    with torch.no_grad():
        for batch in val_loader:
            b_input_ids, b_mask, b_labels = [t.to(device) for t in batch]
            outputs = model(input_ids=b_input_ids, attention_mask=b_mask)
            logits = outputs.logits#ma trận xác suất trước softmax
            preds.extend(torch.argmax(logits, axis=1).cpu().numpy())#danh sách nhãn mô hình dự đoán
            true_labels.extend(b_labels.cpu().numpy())# danh sách nhãn mô hình thực tế

    acc = np.mean(np.array(preds) == np.array(true_labels))# So sánh từng dự đoán với nhãn thật
    print(f" Fold {fold+1} Accuracy: {acc:.4f}")
    accuracies.append(acc)
print("\n Accuracy từng fold:", accuracies)
print(" Accuracy trung bình:", np.mean(accuracies))



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

...

Epoch 1 | Loss: 0.3488


Epoch 2: 100%|██████████| 1250/1250 [03:48<00:00,  5.48it/s]


Epoch 2 | Loss: 0.2063


Epoch 3: 100%|██████████| 1250/1250 [03:48<00:00,  5.48it/s]


Epoch 3 | Loss: 0.1146
 Fold 5 Accuracy: 0.8758

 Accuracy từng fold: [np.float64(0.8834), np.float64(0.8816), np.float64(0.8864), np.float64(0.8836), np.float64(0.8758)]
 Accuracy trung bình: 0.88216


In [19]:
import torch
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
from torch.optim import AdamW
from torch.utils.data import DataLoader, TensorDataset
# Tokenize toàn bộ tập train_df
train_enc = tokenizer(
    list(bert_df['text']),
    truncation=True,
    padding=True,
    max_length=128,
    return_tensors='pt'
)

X_ids = train_enc['input_ids']
X_mask = train_enc['attention_mask']
y = torch.tensor(bert_df['label'].values).long()

train_dataset = TensorDataset(X_ids, X_mask, y)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# Khởi tạo mô hình cuối
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2
)
model.to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)
epochs = 3
total_steps = len(train_loader) * epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

#Huấn luyện full train_df
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        input_ids, attention_mask, labels = [b.to(device) for b in batch]

        optimizer.zero_grad()
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1} | Loss: {total_loss/len(train_loader):.4f}")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1: 100%|██████████| 1563/1563 [04:40<00:00,  5.56it/s]


Epoch 1 | Loss: 0.3378


Epoch 2: 100%|██████████| 1563/1563 [04:45<00:00,  5.48it/s]


Epoch 2 | Loss: 0.2004


Epoch 3: 100%|██████████| 1563/1563 [04:45<00:00,  5.48it/s]

Epoch 3 | Loss: 0.1107


In [20]:
# Lưu mô hình và tokenizer
model.save_pretrained("bert_model_final")
tokenizer.save_pretrained("bert_model_final")

print("\n Đã lưu mô hình ")


 Đã lưu mô hình 
